In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/4.page.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Make No'] = df_input['Make No'].astype(int)
df_input['Year'] = df_input['Year'].astype(int)
df_input['Model No'] = df_input['Model No'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        list_dict_correct_temp = []
        for page in range(1, crawler_series['Page'] + 1):

            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request = f'''{crawler_series['Model Url']}?searching=Y&sort=7&show=300&page={page}'''

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        if html.xpath(f'''//b/a[@href="{crawler_series['Model Url']}"]/text()''')[0] == crawler_series['Model Text']:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    time.sleep(3)
                    continue

            # = = = = = = = = = = = = = = =

            list_url = [href.strip() for href in html.xpath('//div[@class="v-product"]/a[@class="v-product__title productnamecolor colors_productname"]/@href')]

            # = = = = = = = = = = = = = = =

            if not list_url:
                raise

            # = = = = = = = = = = = = = = =

            list_dict_correct_temp.extend([{'Make No': crawler_series['Make No'],
                                            'Make': crawler_series['Make'],
                                            'Year': crawler_series['Year'],
                                            'Model No': crawler_series['Model No'],
                                            'Model': crawler_series['Model'],
                                            'Page': crawler_series['Page'],
                                            'No': i + 1,
                                            'Url': url} for i, url in enumerate(list_url)])

            # = = = = = = = = = = = = = = =

            if count_remain % 30 == 0 and count_remain != 0:
                clear_output()
            print(f'''[第{page}页] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['Make No']}. {crawler_series['Make']} > {crawler_series['Year']} > {crawler_series['Model No']} > {crawler_series['Model']} > {page}. {url_request}\n''')

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.extend(list_dict_correct_temp)
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Make No', 'Year', 'Model No', 'Page', 'No'],
                                                             ascending=[True, False, True, True, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/5.url-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Make No', 'Year', 'Model No', 'Page'],
                                                           ascending=[True, False, True, True],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/url_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[第1页] - [请求1次] - [剩余29条] - [2024/11/25 13:44:27] - 53. Volvo > 2008 > 9 > XC90 > 1. https://www.tpms.com/2008-Volvo-XC90-TPMS-s/9459.htm?searching=Y&sort=7&show=300&page=1

[第1页] - [请求1次] - [剩余28条] - [2024/11/25 13:44:27] - 53. Volvo > 2007 > 6 > V50 > 1. https://www.tpms.com/2007-Volvo-V50-TPMS-s/4887.htm?searching=Y&sort=7&show=300&page=1

[第1页] - [请求1次] - [剩余27条] - [2024/11/25 13:44:28] - 53. Volvo > 2005 > 1 > S40 > 1. https://www.tpms.com/2005-Volvo-S40-TPMS-s/9244.htm?searching=Y&sort=7&show=300&page=1

[第1页] - [请求1次] - [剩余26条] - [2024/11/25 13:44:28] - 53. Volvo > 2005 > 2 > S60 > 1. https://www.tpms.com/2005-Volvo-S60-TPMS-s/4896.htm?searching=Y&sort=7&show=300&page=1

[第1页] - [请求1次] - [剩余25条] - [2024/11/25 13:44:28] - 53. Volvo > 2005 > 5 > V70 > 1. https://www.tpms.com/2005-Volvo-V70-TPMS-s/4898.htm?searching=Y&sort=7&show=300&page=1

[第1页] - [请求1次] - [剩余24条] - [2024/11/25 13:44:28] - 53. Volvo > 2005 > 7 > XC90 > 1. https://www.tpms.com/2005-Volvo-XC90-TPMS-s/4900.htm?search